mnist_cnn 예제에 대한 스터디 내용입니다.

In [2]:
'''Trains a simple convnet on the MNIST dataset.

Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

'Trains a simple convnet on the MNIST dataset.\n\nGets to 99.25% test accuracy after 12 epochs\n(there is still a lot of margin for parameter tuning).\n16 seconds per epoch on a GRID K520 GPU.\n'

import 과정

In [3]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils

Using Theano backend.


batch_size : 한 epoch에서 실행시키는 단위(배치) 크기
nb_classes : 클래스 수
nb_epoch : epoch 수

In [9]:
batch_size = 128
nb_classes = 2
nb_epoch = 12

nb_train_data = 180000
nb_test_data = 36000

# input image dimensions
img_channel = 1
img_row = 32
img_col = 32

# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
nb_pool = 2
# convolution kernel size
nb_conv = 3

데이터 로딩

In [12]:
def load_data(filename, count, channel, row, col):
    print('Loading data from', filename)
    
    fp = open(filename, 'rb')
    buf = fp.read(count * channel * row * col)
    data = np.frombuffer(buf, dtype=np.uint8)
    data = data.reshape(count, channel, row, col)
    
    print('The shape of loaded data is ', data.shape)
    
    return data

In [45]:
def load_labels(filename, count):
    print('Loading labels from ', filename)
    
    fp = open('./seq_model_cnn_data/train_labels.bin', 'r')
    buf = fp.read(count)
    data_bin = []
    for i in buf:
        data_bin.append(i)
    data = np.asarray(data_bin, dtype=np.uint8, order='C')

    print('The shape of loaded labels is ', data.shape)
    
    return data

In [46]:
# the data, shuffled and split between train and test sets
X_train = load_data('./seq_model_cnn_data/train_data.bin', nb_train_data, img_channel, img_row, img_col)
X_test = load_data('./seq_model_cnn_data/test_data.bin', nb_test_data, img_channel, img_row, img_col)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

Loading data from ./seq_model_cnn_data/train_data.bin
The shape of loaded data is  (180000, 1, 32, 32)
Loading data from ./seq_model_cnn_data/test_data.bin
The shape of loaded data is  (36000, 1, 32, 32)


In [47]:
y_train = load_labels('./seq_model_cnn_data/train_labels.bin', nb_train_data)
y_test = load_labels('./seq_model_cnn_data/test_labels.bin', nb_test_data)

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

Loading labels from  ./seq_model_cnn_data/train_labels.bin
The shape of loaded labels is  (180000,)
Loading labels from  ./seq_model_cnn_data/test_labels.bin
The shape of loaded labels is  (36000,)


In [48]:
Y_train

array([[ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       ..., 
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.]])

모델 구축

In [49]:
model = Sequential()
 
model.add(Convolution2D(nb_filters, nb_conv, nb_conv,
                        border_mode='valid',
                        input_shape=(1, img_rows, img_cols)))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, nb_conv, nb_conv))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

compile 및 fit

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_test, Y_test))

Train on 180000 samples, validate on 36000 samples
Epoch 1/12
116096/180000 [==================>...........] - ETA: 138s - loss: 0.2727 - acc: 0.8912

테스트

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])